# Compare fit results to published values

This notebook compares fit results from SNCosmo with fit results published by SDSS and DES.

In [ ]:
import numpy as np
from astropy.table import Table, join
from matplotlib import pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Span, HoverTool, ColumnDataSource
from bokeh.layouts import gridplot

import sys; sys.path.insert(0, '../')
from data_access import sdss_data, des_data

In [ ]:
# Set up bokeh plotting
output_notebook()
_basic_tools = "save,pan,box_zoom,reset,wheel_zoom".split(',')


## Plotting functions

Since we will be creating the same plots for multiple data sets, we create some generic plotting functions. We create a static version of the plot zoomed in to show detail, along with an interactive version that defaults to show all available data.

In [ ]:
def create_static_figure(fit_data, survey_name):
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    line = np.arange(-20, 20)

    for axis, value in zip(axes, ['x0', 'x1', 'c']):
        axis.plot(line, line, linestyle='--', color='grey')
        axis.errorbar(fit_data[f'{value}_{survey_name}'], 
                      fit_data[f'{value}_sncosmo'], 
                      xerr=fit_data[f'{value}_{survey_name}_err'],
                      yerr=fit_data[f'{value}_err_sncosmo'], 
                      linestyle='',
                      alpha=.3)

    axes[3].plot(line, line, linestyle='--', color='grey', alpha=.6)
    axes[3].scatter(fit_data[f'chi_{survey_name}'], fit_data['chi_sncosmo'], alpha=.3, s=10)

    axes[0].set_title('x0')
    axes[0].set_ylabel('SNCosmo Fit ugriz')

    axes[1].set_title('x1')

    axes[2].set_title('c')
    axes[2].set_xlabel('Published Value')
    axes[2].set_ylabel('SNCosmo Fit ugriz')

    axes[3].set_xlabel('Published Value')
    axes[3].set_title('chi2')
    
    return axes


def create_interactive_figure(fit_data, survey_name):
    data_dict = {col.name: np.array(col) for col in fit_data.itercols()}
    source = ColumnDataSource(data=data_dict)

    hover = HoverTool(tooltips=[
            ("target", "@cid"),
            ("class", "@class"),
            ("z", "@z"),
            ('z_fit', '@fit_z'),
            ('chi2', '@chi'),
            ('dof', '@dof'),
        ])

    figures = []
    for value in ('x0', 'x1', 'c', 'chi'):
        fig = figure(tools=_basic_tools + [hover, 'box_select', 'lasso_select'], title=value)
        fig.circle(f'{value}_{survey_name}', f'{value}_sncosmo', source=source, size=4, alpha=.5)
        fig.xaxis.axis_label = 'Published Value'
        fig.yaxis.axis_label = 'SNCosmo Fit ugriz'
        figures.append(fig)
    
    return gridplot(figures, ncols=2, plot_width=350, plot_height=350)



## SDSS

We begin by creating a combined table to store both the published and SNCosmo fit results. Then we plot correlations between both data sets for the fit parameters x0, x1 (stretch), and c (color) along with normalized chi-squared.

In [ ]:
# Get SDSS published data
sdss_published = sdss_data.master_table[
    'CID',
    'x0SALT2zspec', 
    'x0errSALT2zspec', 
    'x1SALT2zspec', 
    'x1errSALT2zspec', 
    'cSALT2zspec', 
    'cerrSALT2zspec',
]

sdss_published.rename_column('CID', 'cid')
sdss_published.rename_column('x0SALT2zspec', 'x0_sdss')
sdss_published.rename_column('x0errSALT2zspec', 'x0_sdss_err')
sdss_published.rename_column('x1SALT2zspec', 'x1_sdss')
sdss_published.rename_column('x1errSALT2zspec', 'x1_sdss_err')
sdss_published.rename_column('cSALT2zspec', 'c_sdss')
sdss_published.rename_column('cerrSALT2zspec', 'c_sdss_err')
sdss_published['chi_sdss'] = (
    sdss_data.master_table['chi2SALT2zspec'] 
    / sdss_data.master_table['ndofSALT2zspec']
)

# Get SNCosmo fit data
sdss_sncosmo = Table.read('sdss_results/snia_ugriz.csv')
sdss_sncosmo.rename_column('x0', 'x0_sncosmo')
sdss_sncosmo.rename_column('x0_err', 'x0_err_sncosmo')
sdss_sncosmo.rename_column('x1', 'x1_sncosmo')
sdss_sncosmo.rename_column('x1_err', 'x1_err_sncosmo')
sdss_sncosmo.rename_column('c', 'c_sncosmo')
sdss_sncosmo.rename_column('c_err', 'c_err_sncosmo')
sdss_sncosmo['chi_sncosmo'] = sdss_sncosmo['chi'] / sdss_sncosmo['dof']
sdss_sncosmo.remove_columns(['chi', 'dof'])

# Combine tables and keep only SN with published fits
combined_sdss = join(sdss_sncosmo, sdss_published)
combined_sdss = combined_sdss[~combined_sdss['x0_sdss'].mask]   
combined_sdss.show_in_notebook(display_length=5)


In [ ]:
# Consider only type 1a objects
fit_data = combined_sdss[combined_sdss['class'] == 'SNIa']
plot_axes = create_static_figure(fit_data, 'sdss')

plot_axes[0].set_xlim(0, .003)
plot_axes[0].set_ylim(0, .003)
plot_axes[1].set_xlim(-3.5, 3.5)
plot_axes[1].set_ylim(-15, 15)
plot_axes[2].set_xlim(-.5, 1)
plot_axes[2].set_ylim(-2, 1.75)
plot_axes[3].set_xlim(0, 8)
plot_axes[3].set_ylim(0, 150)

plt.show()

In [ ]:
plot_grid = create_interactive_figure(fit_data, 'sdss')
show(plot_grid)


## DES

As before, we begin by creating a combined table to store both the published and SNCosmo fit results.

In [ ]:
# Get DES published data
des_published = des_data.master_table[
    'CID',
    'x0',
    'x0ERR',
    'x1',
    'x1ERR',
    'c',
    'cERR'
]

des_published.rename_column('CID', 'cid')
des_published.rename_column('x0', 'x0_des')
des_published.rename_column('x0ERR', 'x0_des_err')
des_published.rename_column('x1', 'x1_des')
des_published.rename_column('x1ERR', 'x1_des_err')
des_published.rename_column('c', 'c_des')
des_published.rename_column('cERR', 'c_des_err')
des_published['chi_des'] = (
   des_data.master_table['FITCHI2'] 
   / des_data.master_table['NDOF']
)


# Get SNCosmo fit data
des_sncosmo = Table.read('des_results/snia_ugriz.csv')
des_sncosmo.rename_column('x0', 'x0_sncosmo')
des_sncosmo.rename_column('x0_err', 'x0_err_sncosmo')
des_sncosmo.rename_column('x1', 'x1_sncosmo')
des_sncosmo.rename_column('x1_err', 'x1_err_sncosmo')
des_sncosmo.rename_column('c', 'c_sncosmo')
des_sncosmo.rename_column('c_err', 'c_err_sncosmo')
des_sncosmo['chi_sncosmo'] = des_sncosmo['chi'] / des_sncosmo['dof']
des_sncosmo['cid'] = [str(x) for x in des_sncosmo['obj_id']]
des_sncosmo.remove_columns(['chi', 'dof', 'obj_id'])

# Combine tables and keep only SN with published fits
combined_des = join(des_sncosmo, des_published)
combined_des.show_in_notebook(display_length=5)


In [ ]:
plot_axes = create_static_figure(combined_des, 'des')

plot_axes[0].set_xlim(0, 5e-4)
plot_axes[0].set_ylim(0, 3e-5)
plot_axes[1].set_xlim(-3.5, 3.5)
plot_axes[1].set_ylim(-5.5, 5.5)
plot_axes[2].set_xlim(-.5, .5)
plot_axes[2].set_ylim(-1.5, 1.75)
plot_axes[3].set_xlim(0, 2)
plot_axes[3].set_ylim(0, 36)

plt.show()


In [ ]:
plot_grid = create_interactive_figure(combined_des, 'des')
show(plot_grid)
